In [1]:
from datasets import load_from_disk
from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer
import numpy as np

# We reload the saved dataset that was preprocessed in dataset-prep notebook.
dataset = load_from_disk('preprocessed-dataset-complete')

# We upload the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("vgaraujov/t5-base-spanish")

# We upload the T5 pretrained model base
model = T5ForConditionalGeneration.from_pretrained('vgaraujov/t5-base-spanish')
#model = MT5ForConditionalGeneration.from_pretrained('./results/t5-finetuned-only-trees')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/franco/Desktop/TESIS/Code/ThesisModel/tesis-env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def token_accuracy(pred):
    logits = pred.predictions
    references = pred.label_ids
    predictions = np.argmax(logits, axis=-1)

    token_accuracies = []
    for pred_tokens, ref_tokens in zip(predictions, references):
        correct_tokens = sum(p == r for p, r in zip(pred_tokens, ref_tokens))
        token_accuracy = correct_tokens / len(ref_tokens) if len(ref_tokens) > 0 else 0
        token_accuracies.append(token_accuracy)

    # Calculate F1-score
    return {
        "token_accuracy": sum(token_accuracies) / len(token_accuracies)
    }

In [11]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    logits = pred.predictions
    references = pred.label_ids
    predictions = np.argmax(logits, axis=-1)

    f1 = f1_score(references, predictions, average="macro")

    return {
        "f1": f1
    }

In [2]:
import torch
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [5]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq


# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',          # Directorio de salida
    eval_strategy='steps',     # Estrategia de evaluación
    learning_rate=3e-4,              # Tasa de aprendizaje
    per_device_train_batch_size=4,   # Tamaño del batch para entrenamiento
    per_device_eval_batch_size=4,    # Tamaño del batch para evaluación
    weight_decay=0.01,               # Decaimiento de peso
    save_total_limit=3,              # Límite de guardado
    num_train_epochs=5,              # Número de épocas de entrenamiento
    fp16=True,                       # Usar FP16 si está disponible
    logging_steps=100,
    logging_dir='./logs'
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Crear el entrenador
trainer = Trainer(
    model=model,                         # El modelo
    args=training_args,                  # Argumentos de entrenamiento
    train_dataset=dataset['train'],    # Dataset de entrenamiento
    eval_dataset=dataset['validation'],# Dataset de validación
    tokenizer=tokenizer,                 # Tokenizador
    data_collator=data_collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

In [6]:
# Entrenar el modelo
trainer.train()

# Guardar el modelo y el tokenizador
trainer.save_model('./results/complete')

  0%|          | 0/4645 [00:00<?, ?it/s]

{'loss': 1.449, 'grad_norm': 1.6775729656219482, 'learning_rate': 0.0002935414424111948, 'epoch': 0.11}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.7685956954956055, 'eval_runtime': 64.1309, 'eval_samples_per_second': 2.214, 'eval_steps_per_second': 0.561, 'epoch': 0.11}
{'loss': 0.6059, 'grad_norm': 1.3278467655181885, 'learning_rate': 0.00028708288482238965, 'epoch': 0.22}


  0%|          | 0/36 [00:00<?, ?it/s]

...

{'eval_loss': 0.13864074647426605, 'eval_runtime': 57.2059, 'eval_samples_per_second': 2.482, 'eval_steps_per_second': 0.629, 'epoch': 4.74}
{'loss': 0.1098, 'grad_norm': 0.8002377152442932, 'learning_rate': 9.364908503767491e-06, 'epoch': 4.84}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.1351364552974701, 'eval_runtime': 80.4721, 'eval_samples_per_second': 1.765, 'eval_steps_per_second': 0.447, 'epoch': 4.84}
{'loss': 0.1004, 'grad_norm': 1.3744999170303345, 'learning_rate': 2.906350914962325e-06, 'epoch': 4.95}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.13297851383686066, 'eval_runtime': 67.7068, 'eval_samples_per_second': 2.097, 'eval_steps_per_second': 0.532, 'epoch': 4.95}
{'train_runtime': 34326.9713, 'train_samples_per_second': 0.541, 'train_steps_per_second': 0.135, 'train_loss': 0.2351704702182273, 'epoch': 5.0}


In [8]:
results = trainer.evaluate(eval_dataset=dataset['test'])
print(results)

  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.10755975544452667, 'eval_runtime': 64.0738, 'eval_samples_per_second': 2.232, 'eval_steps_per_second': 0.562, 'epoch': 5.0}


In [7]:
trainer.state.log_history

[{'loss': 1.449,
  'grad_norm': 1.6775729656219482,
  'learning_rate': 0.0002935414424111948,
  'epoch': 0.10764262648008611,
  'step': 100},
 {'eval_loss': 0.7685956954956055,
  'eval_runtime': 64.1309,
  'eval_samples_per_second': 2.214,
  'eval_steps_per_second': 0.561,
  'epoch': 0.10764262648008611,
  'step': 100},
 {'loss': 0.6059,
  'grad_norm': 1.3278467655181885,
  'learning_rate': 0.00028708288482238965,
  'epoch': 0.21528525296017223,
  'step': 200},
 {'eval_loss': 0.7397180199623108,
  'eval_runtime': 63.608,
  'eval_samples_per_second': 2.232,
  'eval_steps_per_second': 0.566,
  'epoch': 0.21528525296017223,
  'step': 200},
 {'loss': 0.508,
  'grad_norm': 0.9548324346542358,
  'learning_rate': 0.00028062432723358446,
  'epoch': 0.32292787944025836,
  'step': 300},
 {'eval_loss': 0.6178632974624634,
  'eval_runtime': 63.1661,
  'eval_samples_per_second': 2.248,
  'eval_steps_per_second': 0.57,
  'epoch': 0.32292787944025836,
  'step': 300},
 {'loss': 0.4257,
  'grad_norm': 

In [8]:
predictions = trainer.predict(dataset['validation'])

  0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
import numpy as np
np.argmax(predictions.predictions, axis=-1)

In [12]:
compute_metrics(predictions)

ValueError: Found input variables with inconsistent numbers of samples: [281, 2]

In [ ]:
trainer.log_metrics("test", results)